<a href="https://colab.research.google.com/github/Nitin-Dwivedi-7/COVID_19-Test-Prediction-Using-CT-scan-Images/blob/main/Mobilenet_major_25GB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#import Keras packages
import tensorflow as tf
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.preprocessing.image import load_img
import random

import numpy as np
from keras import applications
from keras.layers import Input
from keras.models import Model
from keras import optimizers
from keras.utils import get_file

In [ ]:
import pickle 
file = "/content/drive/My Drive/data_5K.pkl"
f_o=open(file,"rb")
data=pickle.load(f_o)
f_o.close()
file= "/content/drive/My Drive/labels_5k.pkl"
f_o=open(file,"rb")
labels=pickle.load(f_o)
f_o.close()

In [ ]:
data = np.array(data)
labels = np.array(labels)

In [ ]:
img_width, img_height = 256, 256        # Resolution of inputs
#train_data_dir = "train"           # Folder of train samples
#validation_data_dir = "val" # Folder of validation samples44
#nb_train_samples = 10000                # Number of train samples
#nb_validation_samples = 9500            # Number of validation samples
batch_size = 64                        # Batch size
epochs = 20                # Maximum number of epochs

In [ ]:
# Load Mobilenet

model=applications.mobilenet.MobileNet(weights="imagenet", include_top=False, input_shape=(img_width, img_height, 3))



17227776/17225924 [==============================] - 0s 0us/step


In [ ]:
# Freeze first 15 layers
for layer in model.layers[:45]:
	layer.trainable = False
for layer in model.layers[45:]:
   layer.trainable = True

In [ ]:
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
predictions = Dense(2, activation="softmax")(x) # 4-way softmax classifier at the end


In [ ]:
#nb_train_samples = 2543                # Number of train samples
#nb_validation_samples = 2261            # Number of validation samples

# Initializing the CNN


In [ ]:
from keras.utils.np_utils import to_categorical   

categorical_labels = to_categorical(labels, num_classes=2)


from sklearn.model_selection import train_test_split

#X_train1, X_test0, ytrain1, ytest0 = train_test_split(data, categorical_labels, test_size=0.1,
                                                    #random_state=random.randint(0,100))
X_train1 = data
ytrain1 = categorical_labels

for index in range(10):
  classifier = Model(model.input,predictions)
  classifier.compile(loss="categorical_crossentropy", optimizer=optimizers.SGD(lr=1e-3, momentum=0.9), metrics=["accuracy"])


X_train, X_test1, ytrain, ytest1 = train_test_split(X_train1, ytrain1, test_size=0.1,
                                                    random_state=random.randint(0,100))
X_val, X_test, yval, ytest = train_test_split(X_test1, ytest1, test_size=0.5,
                                                    random_state=random.randint(0,100))
    


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
    
train_datagen = ImageDataGenerator(
            rescale=1./255,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True)
    
test_datagen = ImageDataGenerator(rescale=1./255)
    
val_datagen = ImageDataGenerator(rescale=1./255)
    
training_set = train_datagen.flow(
            X_train, ytrain,
            batch_size=64)
    
val_set = val_datagen.flow(
            X_val, yval,
            batch_size=64)
    
test_set = test_datagen.flow(
            X_test, ytest,
            batch_size=64)
    
X, y = test_set.next()
    
classifier.fit_generator(
            training_set,
            steps_per_epoch=20,
            epochs=epochs,
            validation_data=val_set,
            validation_steps=100)
    
    #w_file = 'Wilson_vgg16_model_weights_k10.h5'
    #classifier.save_weights(w_file)
    
    
    

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/20
20/20 [==============================] - 16s 823ms/step - loss: 0.7508 - accuracy: 0.8391 - val_loss: 0.8033 - val_accuracy: 0.8129
Epoch 2/20
20/20 [==============================] - 16s 797ms/step - loss: 0.1394 - accuracy: 0.9688
Epoch 3/20
20/20 [==============================] - 16s 792ms/step - loss: 0.1486 - accuracy: 0.9703
Epoch 4/20
20/20 [==============================] - 16s 799ms/step - loss: 0.1284 - accuracy: 0.9758
Epoch 5/20
20/20 [==============================] - 16s 785ms/step - loss: 0.1066 - accuracy: 0.9787
Epoch 6/20
20/20 [==============================] - 16s 791ms/step - loss: 0.0460 - accuracy: 0.9874
Epoch 7/20
20/20 [==============================] - 16s 790ms/step - loss: 0.0363 - accuracy: 0.9891
Epoch 8/20
20/20 [==============================] - 16s 789ms/step - loss: 0.0429 - accuracy: 0.9882
Epoch 9/20
20/20 [==============================] - 16s 805ms/step - loss:

In [ ]:
arr = classifier.evaluate(X,y)

2/2 [==============================] - 0s 20ms/step - loss: 1.0088e-04 - accuracy: 1.0000


In [ ]:
print(arr)

[0.00010087653936352581, 1.0]


In [ ]:


arr = classifier.predict(X)
arr = np.argmax(arr, axis=1)

print(arr)
print(y)


[1 1 1 0 0 1 1 0 0 0 1 1 1 0 1 0 1 1 1 0 0 0 0 0 0 0 1 0 0 1 0 1 1 0 0 1 1
 1 0 0 0 0 1 1 0 1 1 1 1 1 1 1 1 0 1 0 1 1 0 1 1 0 1 0]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]]


In [ ]:
print(X.shape)

(64, 256, 256, 3)


In [ ]:
print(X[::-1])

[[[[0.1764706  0.1764706  0.1764706 ]
   [0.21960786 0.21960786 0.21960786]
   [0.1764706  0.1764706  0.1764706 ]
   ...
   [0.18823531 0.18823531 0.18823531]
   [0.21176472 0.21176472 0.21176472]
   [0.227451   0.227451   0.227451  ]]

  [[0.19607845 0.19607845 0.19607845]
   [0.1764706  0.1764706  0.1764706 ]
   [0.24705884 0.24705884 0.24705884]
   ...
   [0.1764706  0.1764706  0.1764706 ]
   [0.18039216 0.18039216 0.18039216]
   [0.21176472 0.21176472 0.21176472]]

  [[0.21960786 0.21960786 0.21960786]
   [0.19607845 0.19607845 0.19607845]
   [0.1764706  0.1764706  0.1764706 ]
   ...
   [0.23529413 0.23529413 0.23529413]
   [0.20784315 0.20784315 0.20784315]
   [0.19607845 0.19607845 0.19607845]]

  ...

  [[0.1764706  0.1764706  0.1764706 ]
   [0.1764706  0.1764706  0.1764706 ]
   [0.18431373 0.18431373 0.18431373]
   ...
   [0.25882354 0.25882354 0.25882354]
   [0.18431373 0.18431373 0.18431373]
   [0.1764706  0.1764706  0.1764706 ]]

  [[0.24313727 0.24313727 0.24313727]
   [0.2

In [ ]:
    #array = np.reshape(X,(64,256,256,3)) 


In [ ]:
from PIL import Image
import numpy as np

img = Image.fromarray(array, 'RGB')
img.save('test.png')
img.show()

ValueError: ignored